# Combine
In this notebook the different dataset for rain gauges, hydroestimator predictions and qrnn predictions are combined.

In [1]:
import xarray as xr
import pandas as pd
import os
import datetime

In [2]:
f = open('../path_to_data.txt', "r")
path_to_dataset = os.path.join(f.readline().rstrip("\n"))
f.close() 

In [3]:
pred = xr.open_dataset(os.path.join(path_to_dataset, 'aggregated_rain_gauge_preds', 'preds.nc'))
he = xr.open_dataset(os.path.join(path_to_dataset,'HE','HE_gauges_hour.nc'))
he_corr = xr.open_dataset(os.path.join(path_to_dataset,'HE','HE_gauges_hour_corrected.nc'))

In [4]:
he_combined = xr.merge([he, he_corr], join='inner')
combined = xr.merge([pred, he_combined], join='inner')

In [5]:
real =  xr.open_dataset(os.path.join(path_to_dataset,'real_rain_gauge_data','real_rain_gauge_data.nc'))

In [6]:
combined = xr.merge([combined, real],  join='left')

In [7]:
combined

<xarray.Dataset>
Dimensions:                  (location: 609, time: 738)
Coordinates:
  * location                 (location) int64 0 1 2 3 4 ... 604 605 606 607 608
  * time                     (time) datetime64[ns] 2020-12-01T01:00:00 ... 20...
Data variables:
    xception_posterior_mean  (time, location) float32 ...
    xception_Q0.95           (time, location) float32 ...
    xception_Q0.99           (time, location) float32 ...
    mlp_posterior_mean       (time, location) float32 ...
    mlp_Q0.95                (time, location) float32 ...
    mlp_Q0.99                (time, location) float32 ...
    HE_precip                (time, location) float64 ...
    HE_precip_corr           (time, location) float64 ...
    gauge_precip             (time, location) float64 0.0 nan nan ... nan 0.0

In [9]:
combined.to_netcdf(os.path.join(path_to_dataset,'combined.nc'))